#Installing and Importing Libraries

In [ ]:
## install libraries

!pip install transformers
!pip install sentencepiece

In [ ]:
import pandas as pd
import numpy as np
import random
import json
import regex as re

import matplotlib.pyplot as plt
import os
import sentencepiece
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from transformers import AdamW
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from transformers import XLMRobertaConfig, XLMRobertaModel, XLMRobertaTokenizer

torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")



# Seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

#Data Collection and Compiling

In [ ]:
#This package is given by the competition authorirties for crawling news articles
!pip install semeval_8_2022_ia_downloader

In [ ]:
!pip install Twisted==22.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.6 MB/s eta 0:00:00
  Attempting uninstall: Twisted
    Found existing installation: Twisted 24.3.0
    Uninstalling Twisted-24.3.0:
      Successfully uninstalled Twisted-24.3.0


In [ ]:
!pip install --upgrade numpy scipy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 769.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 53.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into accou

In [ ]:
!pip install "lxml_html_clean"

In [ ]:
#Run this command for downloading and crawling the news aricles from the article links provided in the dataset
!python -m semeval_8_2022_ia_downloader.cli --links_file="input.csv" --dump_dir="output_dir" --download_delay=5

/usr/bin/python3: Error while finding module specification for 'semeval_8_2022_ia_downloader.cli' (ModuleNotFoundError: No module named 'semeval_8_2022_ia_downloader')


In [ ]:
import pandas as pd
training_data = pd.read_csv('input.csv')

training_data[['id_1', 'id_2']] = training_data['pair_id'].str.split('_', expand=True)
training_data = training_data.drop(columns=['pair_id'])
training_data.head()

In [ ]:
import json
import os

# Function to find and read JSON file content by ID
directory_path = 'output_dir_2'

def find_json_text(directory, file_id):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith(file_id) and file.endswith('.json'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as json_file:
                    return json.load(json_file)['text']

def find_json_title(directory, file_id):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith(file_id) and file.endswith('.json'):
                file_path = os.path.join(root, file)
                with open(file_path, 'r') as json_file:
                    return json.load(json_file)['title']

# Add columns to DataFrame to hold the JSON content for each ID
training_data['text_1'] = training_data['id_1'].apply(lambda id_1: find_json_text(directory_path, id_1))
training_data['text_2'] = training_data['id_2'].apply(lambda id_2: find_json_text(directory_path, id_2))

training_data['title_1'] = training_data['id_1'].apply(lambda id_1: find_json_title(directory_path, id_1))
training_data['title_2'] = training_data['id_2'].apply(lambda id_2: find_json_title(directory_path, id_2))


In [ ]:
training_data.to_csv('training_data.csv')

# Data Cleaning

In [ ]:
import pandas as pd

In [ ]:
#Load training data set
originaldata = pd.read_csv('training_data 1.csv', index_col=0) #usecols=['url1_lang', 'url2_lang', 'link1', 'link2','ia_link1', 'ia_link2', 'Geography', 'Entities', 'Time', 'Narrative', 'Overall', 'Style', 'Tone', 'id_1', 'id_2', 'text_1', 'text_2', 'title_1', 'title_2'])

print(originaldata.head())

len(originaldata)
#2944

<ipython-input-17-ebc8c6e4adc6>:2: DtypeWarning: Columns (231,334,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,402) have mixed types. Specify dtype option on import or set low_memory=False.
  originaldata = pd.read_csv('training_data 1.csv', index_col=0) #usecols=['url1_lang', 'url2_lang', 'link1', 'link2','ia_link1', 'ia_link2', 'Geography', 'Entities', 'Time', 'Narrative', 'Overall', 'Style', 'Tone', 'id_1', 'id_2', 'text_1', 'text_2', 'title_1', 'title_2'])


  url1_lang url2_lang                                              link1  \
0        en        en  https://www.washingtonpost.com/local/virginia-...   
1        en        en  https://www.stlucianewsonline.com/guyana-three...   
2        en        en  https://www.teaparty.org/trump-brings-in-2020-...   
3        en        en  https://gadgets.ndtv.com/apps/news/zomato-uber...   
4        en        en  https://news.yahoo.com/india-approves-third-mo...   

                                               link2  \
0  https://www.washingtonpost.com/world/the_ameri...   
1  https://www.thestar.com/news/world/europe/2020...   
2  https://www.timesofisrael.com/trump-says-he-do...   
3  https://gadgets.ndtv.com/internet/news/indian-...   
4  https://www.channelnewsasia.com/news/asia/indi...   

                                            ia_link1  \
0  https://web.archive.org/web/www.washingtonpost...   
1  https://web.archive.org/web/www.stlucianewsonl...   
2  https://web.archive.org/web/www.tea

2944

In [ ]:
#print(originaldata.iloc[:, :19])

data = originaldata.iloc[:, :19]


In [ ]:
#Filter Out Incomplete Data (NA): story ID, URL, title, or text
# Define the required columns
required_columns = ['url1_lang', 'url2_lang', 'link1', 'link2', 'id_1', 'id_2', 'title_1', 'title_2', 'text_1', 'text_2']

# Filter out rows with missing values in any of the required columns
filtered_data = data.dropna(subset=required_columns)

# Display the filtered data to ensure it’s correctly filtered
filtered_data.head()

len(filtered_data)
#1582

1582

In [ ]:
#Filter Out Same Stories: articles that have titles or URLs that exactly match a newer news article

# Define the columns to check for duplicates (link1, link2, title_1, title_2)
duplicate_columns = ['link1', 'link2', 'title_1', 'title_2']

# Drop duplicate rows based on the specified columns, keeping the first occurrence
filtered_data_no_duplicates = filtered_data.drop_duplicates(subset=duplicate_columns, keep='first')

# Display the cleaned dataset
filtered_data_no_duplicates.head()

#len(filtered_data_no_duplicates)
#1580

,url1_lang,url2_lang,link1,link2,ia_link1,ia_link2,Geography,Entities,Time,Narrative,Overall,Style,Tone,id_1,id_2,text_1,text_2,title_1,title_2
3,en,en,https://gadgets.ndtv.com/apps/news/zomato-uber...,https://gadgets.ndtv.com/internet/news/indian-...,https://web.archive.org/web/gadgets.ndtv.com/a...,https://web.archive.org/web/gadgets.ndtv.com/i...,1,2.333333333,2.666666667,1.666666667,2,1.666666667,1.666666667,1576314516,1576455088,Uber has sold its online food-ordering busines...,Rapid digitisation and growth in both online b...,Zomato Buys Uber's Food Delivery Business in I...,Indian Online Food Delivery Market to Hit $8 B...
5,en,en,https://jewishjournal.com/news/nation/309033/w...,https://jewishjournal.com/online/309000/jewish...,https://web.archive.org/web/jewishjournal.com/...,https://web.archive.org/web/jewishjournal.com/...,1.25,1.75,1.25,1.75,2,1,1.25,1484189120,1484113136,The Simon Wiesenthal Center called on the Whit...,Jewish groups have expressed shock and horror ...,Wiesenthal Center Calls for FBI Task Force to ...,Jewish Groups React to Monsey Stabbings
6,en,en,https://www.financialexpress.com/market/commod...,https://www.financialexpress.com/india-news/de...,https://web.archive.org/web/www.financialexpre...,https://web.archive.org/web/www.financialexpre...,3,4,1.333333333,4,3.666666667,1.333333333,1.333333333,1484034982,1483785560,The government on Wednesday slashed import dut...,Delhiites continued to remain in grip of cold ...,Big cut on import duty on crude and refined pa...,Delhi weather update: Delhiites shiver at 2.4 ...
7,en,en,https://www.opednews.com/articles/The-biggest-...,https://nypost.com/2019/12/31/the-lefts-consta...,https://web.archive.org/web/www.opednews.com/a...,https://web.archive.org/web/nypost.com/2019/12...,2.333333333,3.666666667,1.333333333,3.666666667,3.333333333,1.333333333,1.333333333,1484188439,1484378177,"From The Guardian From Boeing to Whole Foods,...",Police have arrested a 38-year-old black man f...,The biggest business con of 2019: fleecing wor...,The leftâ€™s constant victimhood olympics is a...
8,en,en,https://www.amazon.com/dp/154173016X/?tag=slat...,https://www.amazon.com/dp/1610399579/?tag=slat...,https://web.archive.org/web/www.amazon.com/dp/...,https://web.archive.org/web/www.amazon.com/dp/...,1,4,4,1,4,1,1,1484011751,1483920335,Review CrimeReads best nonfiction crime book ...,"Enter the characters you see below Sorry, we ...",The Compatriots: The Brutal and Chaotic Histor...,Amazon.com


In [ ]:
#Filter URL links

# Remove the columns with url links
data_cleaned = filtered_data_no_duplicates.drop(columns=['link1', 'link2', 'ia_link1', 'ia_link2'])

# Display the cleaned dataset without link1 and link2 columns
data_cleaned.head()

#len(data_cleaned)
#1580

,url1_lang,url2_lang,Geography,Entities,Time,Narrative,Overall,Style,Tone,id_1,id_2,text_1,text_2,title_1,title_2
3,en,en,1,2.333333333,2.666666667,1.666666667,2,1.666666667,1.666666667,1576314516,1576455088,Uber has sold its online food-ordering busines...,Rapid digitisation and growth in both online b...,Zomato Buys Uber's Food Delivery Business in I...,Indian Online Food Delivery Market to Hit $8 B...
5,en,en,1.25,1.75,1.25,1.75,2,1,1.25,1484189120,1484113136,The Simon Wiesenthal Center called on the Whit...,Jewish groups have expressed shock and horror ...,Wiesenthal Center Calls for FBI Task Force to ...,Jewish Groups React to Monsey Stabbings
6,en,en,3,4,1.333333333,4,3.666666667,1.333333333,1.333333333,1484034982,1483785560,The government on Wednesday slashed import dut...,Delhiites continued to remain in grip of cold ...,Big cut on import duty on crude and refined pa...,Delhi weather update: Delhiites shiver at 2.4 ...
7,en,en,2.333333333,3.666666667,1.333333333,3.666666667,3.333333333,1.333333333,1.333333333,1484188439,1484378177,"From The Guardian From Boeing to Whole Foods,...",Police have arrested a 38-year-old black man f...,The biggest business con of 2019: fleecing wor...,The leftâ€™s constant victimhood olympics is a...
8,en,en,1,4,4,1,4,1,1,1484011751,1483920335,Review CrimeReads best nonfiction crime book ...,"Enter the characters you see below Sorry, we ...",The Compatriots: The Brutal and Chaotic Histor...,Amazon.com


In [ ]:
data_cleaned.to_csv('cleaned_training_data.csv')

NameError: name 'data_cleaned' is not defined

In [ ]:
#Added Data Cleaning

#Remove Line Breaks and Tabs

# Specify the columns where you want to remove line breaks
columns_to_clean = ['text_1', 'text_2']

# Remove line breaks (\n and \r) from the specified columns
for column in columns_to_clean:
    data_cleaned[column] = data_cleaned[column].str.replace(r'[\n\r\t]', ' ', regex=True)

# Display the cleaned dataset
print(data_cleaned[columns_to_clean].head())

                                              text_1  \
3  Uber has sold its online food-ordering busines...   
5  The Simon Wiesenthal Center called on the Whit...   
6  The government on Wednesday slashed import dut...   
7  From The Guardian  From Boeing to Whole Foods,...   
8  Review  CrimeReads best nonfiction crime book ...   

                                              text_2  
3  Rapid digitisation and growth in both online b...  
5  Jewish groups have expressed shock and horror ...  
6  Delhiites continued to remain in grip of cold ...  
7  Police have arrested a 38-year-old black man f...  
8  Enter the characters you see below  Sorry, we ...  


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save the file to your Google Drive (adjust the path as needed)
data_cleaned.to_csv('/content/drive/My Drive/cleaned_training_data_2.csv', index=False)

Mounted at /content/drive


In [ ]:
#English (10M articles), Spanish (4.6M), Russian (1.8M), German (1.3M), French (1.2M), Arabic (1.8M), Italian (1.5M), Turkish (655K), Polish (369K), and Mandarin Chinese (205K)

In [ ]:
#Training data v0.1: This release consists of 2,939 pairs of news articles from the following languages (with counts): en: 1761, es: 444, de: 414, pl: 186, tr: 54, ar: 52, and fr: 28. All pairs have the same language in this release.

#Training and Experimenting with XLM-Roberta

In [ ]:
import pandas as pd
training_data = pd.read_csv('/content/drive/My Drive/cleaned_training_data_2.csv')
training_data = training_data.dropna()
training_data

In [ ]:
def merge_clean_columns(df):
    """
    Merge text and title columns into one
    """
    df['merge1'] = df['text_1'].astype(str) + ', ' \
        + df['title_1'].astype(str)

    df['merge2'] = df['text_2'].astype(str) + ', ' \
        + df['title_2'].astype(str)

    return df

In [ ]:
processed_data = merge_clean_columns(training_data)
# split into train and validation
train, dev = train_test_split(processed_data, test_size=0.1, random_state = 42)

In [ ]:

## set parameters
max_len = 512
batch_size = 5
lr = 5e-6
weight_decay = 1e-4
num_epochs = 8

In [ ]:

def get_data_loader(data, batch_size_flg = True):
  """
   Tokenizing dataset and creating dataloader for pytorch format
    """
  tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
  input_ids, attention_masks, labels = [], [], []
  for idx, row in data.iterrows():
      text1, text2 = row['merge1'], row['merge2']
      encode_dict = tokenizer(text1,text2,
                                  max_length=max_len,
                                  padding='max_length',
                                  truncation=True,
                                  add_special_tokens=True
                                  )

      input_ids.append(encode_dict['input_ids'])
      attention_masks.append(encode_dict['attention_mask'])
      # model is used to predict all labels?? -> should we convert to only 1 label
      labels.append([float(x) for x in [row['Geography'],row['Entities'],row['Time'],row['Narrative'],row['Overall'],row['Style'],row['Tone']]])

  input_ids = torch.tensor(input_ids)
  attention_masks = torch.tensor(attention_masks)
  labels = torch.tensor(labels)

  data = TensorDataset(input_ids, attention_masks, labels)
  if(batch_size_flg):
      data_loader = DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
  else:
      data_loader = DataLoader(data)
  return data_loader



In [ ]:
train_data_loader = get_data_loader(train)
eval_data_loader = get_data_loader(dev, False)

In [ ]:
#Adding customized layers on top of pretrained Roberta Model for fine-tuning
class Custom_XLMRoberta(nn.Module):
    def __init__(self, model, hidden_size):
        super(Custom_XLMRoberta, self).__init__()
        self.reg_model = model
        self.fc1 = nn.Linear(hidden_size, 512)
        # self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(512, 100)
        self.fc3 = nn.Linear(100, 7) ## currently processes the 7 labels that we have defined for 7 output types
        self.activation1 = nn.GELU()
        self.activation2 = nn.GELU()

    def forward(self, input_ids, attention_masks):
        output1 = self.reg_model(input_ids, attention_masks)[1]
        # output2 =
        # x = self.dropout(x)
        # logits1= s
        logits1= self.fc3(self.activation2(self.fc2(self.activation1(self.fc1(output1)))))

        return logits1

In [ ]:
import numpy as np
from scipy.stats import pearsonr

def calculate_pearson_per_aspect(predictions, labels):
    """
    Calculates Pearson correlation for each similarity aspect (output dimension).

    Returns:
    - correlations: List of Pearson correlation coefficients for each aspect.
    """
    correlations = []
    predictions = np.array(predictions)
    labels = np.array(labels)
    for i in range(predictions.shape[1]):  # Loop through each of the 7 output scores
        r, _ = pearsonr(predictions[:, i], labels[:, i])
        correlations.append(r)
    return correlations






In [ ]:
train_losses=[]
val_losses =[]
pearson_scores = []

#Evaluation on Validation dataset
def evaluate(model, data_loader,criterion):
  model.eval()
  overall_pred, overall_true = [], []
  with torch.no_grad():
    val_loss_sum=0
    for idx, (ids, att_msks, y) in enumerate(data_loader):
      ids, att_msks, y = ids.to(device), att_msks.to(device), y.to(device)
      y_pred = model(ids, att_msks)
      loss = criterion(torch.squeeze(y_pred),torch.squeeze(y))
      val_loss_sum += loss.item()

      y_pred, y = torch.squeeze(y_pred).cpu().numpy().tolist(), torch.squeeze(y).cpu().numpy().tolist()
      overall_pred.append(y_pred)
      overall_true.append(y)
  val_losses.append(val_loss_sum/len(data_loader))
  return overall_pred, overall_true

#experimentation using weighted loss for each output aspect
# def weighted_loss( y_pred, y, criterion, loss_weights):
#   loss = 0.0
#   for i in range(7):
#     y_pred_i, y_i = y_pred[:, i], y[:, i]
#     loss += criterion(y_pred_i, y_i) * loss_weights[i]
#   return loss


def train(model, model_path, train_data_loader, eval_data_loader, optimizer, loss_weights, epochs):
  model.train()

  #Using MSE loss for gradient descent
  criterion = nn.MSELoss()
  best_pearson = 0
  for i in range(epochs):
    train_loss_sum = 0
    for idx, (ids, att_msks, y) in enumerate(train_data_loader):
      print(idx)
      ids, att_msks, y = ids.to(device), att_msks.to(device), y.to(device)
      optimizer.zero_grad()
      y_pred = model(ids, att_msks)
      y_pred, y = torch.squeeze(y_pred), torch.squeeze(y)
      # loss = weighted_loss(y_pred, y, criterion, loss_weights)
      print(y_pred)
      loss = criterion(y_pred,y)
      loss.backward()
      optimizer.step()
      train_loss_sum += loss.item()

    train_losses.append(train_loss_sum/len(train_data_loader))
    print(f"Loss at epoch {i}: {train_loss_sum:.4f}")

    ## Determine best epoch model using correlation coefficient for Overall in dev data
    eval_pred_overall, eval_true_overall = evaluate(model, eval_data_loader,criterion)

    pearson_correlations = calculate_pearson_per_aspect(eval_pred_overall, eval_true_overall)
    for i, r in enumerate(pearson_correlations):
        print(f"Pearson correlation for aspect {i+1}: {r:.4f}")

    #calculate the mean Pearson correlation
    curr_pearson = np.mean(pearson_correlations)
    print(f"Mean Pearson correlation: {curr_pearson:.4f}")
    pearson_scores.append(curr_pearson)


    #save model only if current pearson score is better than the best
    if curr_pearson > best_pearson:
      best_pearson = curr_pearson
      torch.save(model.state_dict(), model_path)



In [ ]:
#Using GPU for training
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")



In [ ]:
#Initializing pretrained model
pre_trained_model = XLMRobertaModel.from_pretrained("xlm-roberta-base")
config = XLMRobertaConfig.from_pretrained("xlm-roberta-base")
hidden_size = config.hidden_size
# hidden_size = 768
overall_weight = 0.25
loss_weights = [overall_weight if i == 4 else (1-overall_weight)/6 for i in range(7)]

model = Custom_XLMRoberta(pre_trained_model, hidden_size)
model.to(device)

model_path = "XLM_Roberta_0.25_overall_loss.pth"


In [ ]:
#Initializing optimizer and training the model
optimizer = AdamW(model.parameters(), lr=lr, weight_decay = weight_decay)
train(model, model_path, train_data_loader, eval_data_loader, optimizer, loss_weights, num_epochs)





In [ ]:
import matplotlib.pyplot as plt

# Plot the loss curves
plt.plot(range(1, num_epochs + 1), train_losses, marker='o', label='Training Loss')

plt.plot(range(1, num_epochs + 1), val_losses, marker='o', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Avg Loss per batch')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Plot the pearson score curves
plt.plot(range(1, num_epochs + 1), pearson_scores, marker='o', label='Pearson Scores')


plt.xlabel('Epoch')
plt.ylabel('Overall Pearson Scores')
plt.title('Pearson Scores over Epochs')
plt.legend()
plt.show()

In [ ]:
#Evaluation on test data
def predict(model, data_loader):
  model.eval()
  overall_pred, overall_true = [], []
  with torch.no_grad():
    val_loss_sum=0
    for idx, (ids, att_msks, y) in enumerate(data_loader):
      ids, att_msks, y = ids.to(device), att_msks.to(device), y.to(device)
      y_pred = model(ids, att_msks)


      y_pred, y = torch.squeeze(y_pred).cpu().numpy().tolist(), torch.squeeze(y).cpu().numpy().tolist()
      overall_pred.append(y_pred)
      overall_true.append(y)

  return overall_pred, overall_true


In [ ]:
#Loading test set
test_data = pd.read_csv('testing_set.csv')
test_data = test_data[test_data['lang1']==test_data['lang2']]
test_data = test_data.rename(columns={'text_1': 'merge1', 'text_2': 'merge2'})
test_data_loader = get_data_loader(test_data,False)

#Loading best saved model
model.load_state_dict(torch.load("XLM_Roberta_0.25_overall_loss.pth"), strict=False)
model.to(device)
test_pred_overall, test_true_overall = predict(model, test_data_loader)
print(test_pred_overall)


In [ ]:
#Calculating pearson score for test set
pearson_correlations = calculate_pearson_per_aspect(test_pred_overall, test_true_overall)
for i, r in enumerate(pearson_correlations):
        print(f"Pearson correlation for aspect {i+1}: {r:.4f}")

curr_pearson = np.mean(pearson_correlations)
print(f"Mean Pearson correlation for test dataset: {curr_pearson:.4f}")

In [ ]:
test_pred_overall